### Задача предоставлена партнером олимпиады — Федеральным исследовательским центром "Информатика и управление" РАН

        На некотором несуществующем интернет-ресурсе "Политическая кухня" популярностью пользуются два вида видеороликов: про политику и про кулинарию. При этом под роликами про консервативную политику комментарии оставляют только консерваторы, про либеральную --- только либералы. Кулинарные видео комментируют только кулинары. Иногда на ресурсе "Политическая кухня" происходит сбой, и комментарии перепутываются (кулинарный комментарий попадает под политическое видео, либеральный --- под консервативное видео и т.п.), тогда необходимо по комментарию определить, кто его оставил (консерватор, либерал или кулинар) и перенести в соответствующий раздел. Доступа к самим текстам комментариев у команды "Политической кухни" нет, но все тексты прошли обработку лингвистическим анализатором и каждый представлен набором численных признаков.
    
    

> Перед вами стоит задача разработать алгоритм машинного обучения, предсказывающий кем был написан комментарий: консерватором, либералом или кулинаром.

### Формат ввода

[Набор данных](https://disk.yandex.ru/d/9XuZIF8IIAx2fw)


Тренировочная выборка Train.csv представляет собой csv-таблицу со столбцами-признаками и столбцом целевой переменной target.

Описание признаков обучающих данных:

• ___comments_count___ – общее количество комментариев под видео, для которого создан комментарий,

• ___replies_count___ – общее количество ответов на комментарии под видео,

• ___both_count___ – общее количество сообщений под видео,

• ___sentence_count___ – общее количество предложений под видео,

• ___word_count___ – общее количество слов под видео,

• ___target___ – метка кем был оставлен комментарий (0 – либерал, 1 – кулинар, 2 – консерватор),

• ___остальные столбцы___ – признаки, полученные с помощью лингвистического анализатора.

Тестовая выборка Test.csv представляет собой csv-таблицу со столбцами-признаками --- данные, ответы на которые нужно послать в тестирующую систему.

### Формат вывода
В файле SampleSubmission.csv показан формат файла ответа, который нужно загружать в систему. В связи с особенностями работы скрипта подсчета метрики качества, необходимо сохранять порядок следования строк в выходном файле submission.csv.

>Примечания:
Используемая метрика --- многоклассовый F1-score с глобальным усреднением.

Ваш балл вычисляется по формуле 
f1⋅10, где f1 --- результат работы вашего алгоритма.
______


######  1. Импорт библиотек

In [1]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

In [2]:
def  Export(filename,y_pred):
    df_export=pd.read_csv('data/SampleSubmission.csv')
    df_export['target'] = y_pred
    df_export.to_csv('out/'+filename+'.csv', sep=',',index=False)


###### 2. Импортируем тренировочный и тестовый датафрейм, и выведем оглавление таблиц

In [3]:
df_test = pd.read_csv('data/Test.csv')
df_train = pd.read_csv('data/Train.csv')

In [4]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1449 entries, 0 to 1448
Data columns (total 24 columns):
 #   Column                                       Non-Null Count  Dtype
---  ------                                       --------------  -----
 0   comments_count                               1449 non-null   int64
 1   replies_count                                1449 non-null   int64
 2   both_count                                   1449 non-null   int64
 3   sentence_count                               1449 non-null   int64
 4   word_count                                   1449 non-null   int64
 5   basic_emotions                               1449 non-null   int64
 6   basic emotions_fear                          1449 non-null   int64
 7   basic_emotions_anger                         1449 non-null   int64
 8   basic_emotions_sadness                       1449 non-null   int64
 9   basic_emotions_joy                           1449 non-null   int64
 10  basic_emotions_surprise_

In [5]:
df_test.describe()

,comments_count,replies_count,both_count,sentence_count,word_count,basic_emotions,basic emotions_fear,basic_emotions_anger,basic_emotions_sadness,basic_emotions_joy,...,emotions_of_social_evaluation_shame,emotions_of_social_evaluation_guilt,emotions_of_social_evaluation_embarrassment,emotions_of_social_evaluation_pride,emotions_of_social_evaluation_envy,state,state_excited,state_rest,state_estimate,state_process_execution
count,1449.000000,1449.000000,1449.000000,1449.000000,1.449000e+03,1449.000000,1449.000000,1449.000000,1449.000000,1449.000000,...,1449.000000,1449.000000,1449.000000,1449.000000,1449.000000,1449.000000,1449.000000,1449.000000,1449.000000,1449.000000
mean,1215.049689,545.133195,1760.182885,1877.486542,1.638734e+04,321.208420,28.761905,16.770186,70.017943,101.520359,...,4.895790,2.977916,1.819186,2.371981,1.318150,149.850932,17.981366,122.601104,0.536232,8.732229
std,13920.010027,4974.527210,18853.504819,19024.548190,1.531063e+05,3333.997187,376.616407,139.917454,984.441194,786.874788,...,61.400175,23.120022,22.019746,21.726173,10.107787,1496.692402,193.950880,1228.800170,3.020230,74.595577
min,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,36.000000,6.000000,46.000000,67.000000,6.050000e+02,11.000000,0.000000,0.000000,1.000000,3.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,0.000000,4.000000,0.000000,0.000000
50%,116.000000,60.000000,194.000000,212.000000,2.172000e+03,36.000000,2.000000,2.000000,5.000000,11.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,17.000000,2.000000,14.000000,0.000000,1.000000
75%,432.000000,251.000000,711.000000,764.000000,6.774000e+03,123.000000,6.000000,7.000000,17.000000,42.000000,...,1.000000,1.000000,0.000000,1.000000,0.000000,61.000000,6.000000,50.000000,0.000000,4.000000
max,376029.000000,129164.000000,505193.000000,500449.000000,3.880182e+06,87515.000000,9950.000000,3815.000000,29824.000000,19716.000000,...,1731.000000,539.000000,699.000000,548.000000,209.000000,39637.000000,5246.000000,32440.000000,60.000000,1904.000000


In [6]:
print(df_train)

      comments_count  replies_count  both_count  sentence_count  word_count  \
0                152            223         375             514        6682   
1                306            129         435             488        4550   
2                 89             13         102             180        1373   
3                193             87         280             336        2509   
4                509             26         535             969        7076   
...              ...            ...         ...             ...         ...   
3376             830             77         907            1578       11310   
3377              30              0          30              36         334   
3378              69             10          79             131        1209   
3379             252            180         432             567        5922   
3380             732            438        1170            1231       11797   

      basic_emotions  basic emotions_fear  basic_em

In [7]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3381 entries, 0 to 3380
Data columns (total 25 columns):
 #   Column                                       Non-Null Count  Dtype
---  ------                                       --------------  -----
 0   comments_count                               3381 non-null   int64
 1   replies_count                                3381 non-null   int64
 2   both_count                                   3381 non-null   int64
 3   sentence_count                               3381 non-null   int64
 4   word_count                                   3381 non-null   int64
 5   basic_emotions                               3381 non-null   int64
 6   basic emotions_fear                          3381 non-null   int64
 7   basic_emotions_anger                         3381 non-null   int64
 8   basic_emotions_sadness                       3381 non-null   int64
 9   basic_emotions_joy                           3381 non-null   int64
 10  basic_emotions_surprise_

In [8]:
df_train.describe()

,comments_count,replies_count,both_count,sentence_count,word_count,basic_emotions,basic emotions_fear,basic_emotions_anger,basic_emotions_sadness,basic_emotions_joy,...,emotions_of_social_evaluation_guilt,emotions_of_social_evaluation_embarrassment,emotions_of_social_evaluation_pride,emotions_of_social_evaluation_envy,state,state_excited,state_rest,state_estimate,state_process_execution,target
count,3381.000000,3381.000000,3381.000000,3381.000000,3.381000e+03,3381.000000,3381.000000,3381.000000,3381.000000,3381.00000,...,3381.000000,3381.000000,3381.000000,3381.000000,3381.000000,3381.000000,3381.000000,3381.000000,3381.000000,3381.000000
mean,716.322094,389.283644,1105.605738,1175.560485,1.065012e+04,193.550133,14.044957,12.559006,34.935226,68.86572,...,2.174209,1.125702,1.572612,1.073943,94.000887,11.471162,76.454599,0.419403,5.655723,1.432121
std,4299.374831,2296.896085,6520.227387,6664.666222,5.977782e+04,1119.565542,98.824360,66.774307,231.774475,400.77136,...,14.517021,7.772497,9.579771,8.893710,521.524465,66.381813,431.285580,1.879135,26.564209,0.598240
min,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,36.000000,6.000000,45.000000,69.000000,6.230000e+02,11.000000,0.000000,0.000000,1.000000,3.00000,...,0.000000,0.000000,0.000000,0.000000,5.000000,0.000000,4.000000,0.000000,0.000000,1.000000
50%,107.000000,53.000000,172.000000,203.000000,2.003000e+03,34.000000,2.000000,2.000000,5.000000,10.00000,...,0.000000,0.000000,0.000000,0.000000,16.000000,2.000000,12.000000,0.000000,1.000000,1.000000
75%,408.000000,250.000000,691.000000,702.000000,6.342000e+03,112.000000,6.000000,7.000000,17.000000,37.00000,...,1.000000,0.000000,1.000000,0.000000,56.000000,6.000000,45.000000,0.000000,4.000000,2.000000
max,136877.000000,91912.000000,228789.000000,227434.000000,2.107711e+06,42833.000000,2856.000000,1801.000000,9254.000000,16315.00000,...,370.000000,257.000000,262.000000,376.000000,16778.000000,2317.000000,13539.000000,48.000000,874.000000,2.000000


In [9]:
print(df_test)

      comments_count  replies_count  both_count  sentence_count  word_count  \
0                196            121         317             425        4026   
1                 59             12          71             187        1698   
2                 44              0          44              54         393   
3                664            278         942            1296       10235   
4                 11              0          11              57        1275   
...              ...            ...         ...             ...         ...   
1444              59              0          59              83         589   
1445             109             37         146             167        2047   
1446              74             31         105             184        2165   
1447            2098            389        2487            4090       24219   
1448              44              0          44              47         304   

      basic_emotions  basic emotions_fear  basic_em

#### Обработка датафрейма

### метрика


In [10]:
from sklearn.utils import shuffle


df_train_metric = shuffle(df_train)

df_train_metricTrain = df_train_metric.sample(frac=0.7)

df_train_metricTrainX = df_train_metricTrain.drop('target', axis=1)
df_train_metricTrainY = df_train_metricTrain['target']

df_train_metricTest = df_train_metric.drop(df_train_metricTrain.index)

df_train_metricTest_Y = df_train_metricTest['target']
df_train_metricTest_X = df_train_metricTest.drop('target', axis=1)


### КНН

In [11]:
from sklearn.metrics import f1_score
from sklearn.neighbors import KNeighborsClassifier 

for i in range(1,2):
    kNN = KNeighborsClassifier(n_neighbors=i) 
    kNN.fit(df_train_metricTrainX, df_train_metricTrainY)

    predMetric = kNN.predict(df_train_metricTest_X)
    
    print(str(i)+" "+ str(f1_score(df_train_metricTest_Y.values, predMetric, average='micro')))

print(predMetric)
print(df_train_metricTest_Y.values)

1 0.722879684418146
[2 1 1 ... 1 1 1]
[1 1 2 ... 1 1 1]


Классификатор повышения градиента

In [12]:
from sklearn.ensemble import GradientBoostingClassifier

for i in range(1,2):
    clf = GradientBoostingClassifier(n_estimators=1000, learning_rate=0.2,max_depth=1, random_state=0).fit(df_train_metricTrainX, df_train_metricTrainY)
    predMetric = clf.predict(df_train_metricTest_X)
    
    print(str(i)+" "+ str(f1_score(df_train_metricTest_Y.values, predMetric, average='micro')))

print(predMetric)
print(df_train_metricTest_Y.values)

1 0.849112426035503
[2 1 2 ... 2 1 1]
[1 1 2 ... 1 1 1]


LGBM Classifier

In [13]:
#from lightgbm import LGBMClassifier

Классификатор XGBoost

In [14]:
#from xgboost.sklearn import XGBClassifier

Случайный Forest

In [15]:
from sklearn.ensemble import RandomForestClassifier

Древо решений

In [16]:
from sklearn.tree import DecisionTreeClassifier

Классификатор стохастического градиентного спуска

In [17]:
from sklearn.linear_model import SGDClassifier

Наивный байесовский (гауссовский, полиномиальный)

In [18]:
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB

Машина опорных векторов

In [19]:
from sklearn.svm import SVC

Логистическая регрессия

In [20]:
from sklearn.linear_model import LogisticRegression

##### Предсказание 

In [21]:
df_train_Y = df_train['target']
df_train_X = df_train.drop('target', axis=1)

print(df_train_Y)
print(df_train_X)

0       2
1       2
2       1
3       1
4       1
       ..
3376    1
3377    1
3378    2
3379    2
3380    0
Name: target, Length: 3381, dtype: int64
      comments_count  replies_count  both_count  sentence_count  word_count  \
0                152            223         375             514        6682   
1                306            129         435             488        4550   
2                 89             13         102             180        1373   
3                193             87         280             336        2509   
4                509             26         535             969        7076   
...              ...            ...         ...             ...         ...   
3376             830             77         907            1578       11310   
3377              30              0          30              36         334   
3378              69             10          79             131        1209   
3379             252            180         432            

In [22]:
df_test_X = df_test

In [23]:
kNN = KNeighborsClassifier(n_neighbors=11) 
kNN.fit(df_train_X, df_train_Y) 

y_pred = kNN.predict(df_test_X)
print(y_pred)
#print(kNN.predict_proba(df_test_XY)) 

Export('submission', y_pred)

[2 2 1 ... 2 1 1]


In [24]:


for i in range(1,2):
    clf = GradientBoostingClassifier(n_estimators=1000, learning_rate=0.2,max_depth=1, random_state=0).fit(df_train_X, df_train_Y)
    y_pred = clf.predict(df_test_X)
    

Export('submission', y_pred)